<a href="https://colab.research.google.com/github/wafibismail/csp-scheduling/blob/main/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Problem Description

In this particular CSP, these are the constraints

| Constraint Type | Example/s |
|-|-|
|Completeness|All courses must have an assigned schedule in the timetable|
|Conflict of resources|A Professor cannot attend two classes at the same time|
|-|A room cannot be used by two classes at the same time|
|Maximum capacity|Assignment of courses into rooms must consider the capacity of classrooms and other facilities.
|Availability of resources|Professors and rooms must not be assigned to classes on days they are unavailable|
|Scheduling patterns|Classes cannot begin earlier than the earliest allowable time and cannot end beyond the latest allowable time.
|-|There should be no instances of single classes in a day or too many of which.
|-|The distance between the rooms assigned to back-to-back classes should be minimal to avoid unreasonable travel.
|-|Courses should not be scheduled for evening sessions.
|-|No classes must be held on weekends.

I try to run the current partial solution with a sample data, but no luck, time complexity too high even for the small dataset

The current solution is too basic, akin to BFS. I intend to sort of start from square one (not completely) making use of the greedy search concept. First need to construct the path cost function (will continue this tomorrow, i sleep now)

Performance measures to be interpreted as heuristic in a tree traversal algorithm to meet the above constraints:
<table align=center>
  <tr>
    <th>Performance measure</th>
    <th>Objective</th>
    <th>Priority</th>
  </tr>
  <tr>
    <td>COMPLETENESS (all classes assigned to a slot)</td>
    <td>Satisfy<br>(Prune)</td>
    <td>Highest</td>
  </tr>
  <tr>
    <td>
      AVAILABILITY of VENUE considered<br><br>
      Appeases:<br>
      Venues not assigned in days they are not available
    </td>
    <td>Satisfy<br>(Prune)</td>
    <td>Highest</td>
  </tr>
  <tr>
    <td>
      Only ALLOWABLE TIMESLOTS allocated<br><br>
      Appeases:<br>
      Scheduling pattern: No classes held on agreed-on unavailable times i.e., weekends
    </td>
    <td>Satisfy</td>
    <td>Highest</td>
  </tr>
    <td>
      More PREFERRED TIMESLOTS allocated<br><br>
      Appeases:<br>
      Scheduling pattern: Less classes held on less prefered times, e.g. Wednesday pm & Fridays
    </td>
    <td>Maximize</td>
    <td>High</td>
  </tr>
  <tr>
    <td>
      AVAILABILITY of TEACHER considered<br><br>
      Appeases:<br>
      Teachers not assigned in days they are not available
    </td>
    <td>Satisfy</td>
    <td>Highest</td>
  </tr>
  <tr>
    <td>
      FACILITY requirement (of a course) = available in venue<br><br>
      Appeases:<br>
      Courses not assigned to classes with insufficient facilities</td>
    <td>Satisfy</td>
    <td>Highest</td>
  </tr>
  <tr>
    <td>
      QUOTA difference, i.e., difference between a course's capacity and venue's capacity<br>
      <br>
      Appeases:<br>
      Maximum capacity (reduces likelihood for any course to not be assigned sufficiently large room)
    </td>
    <td>Minimize</td>
    <td>Highest</td>
  </tr>
  <tr>
    <td>
      SPREAD of a VENUE's allocation throughout all available times<br>
      <br>
      Appeases:<br>
      1. Schedule pattern: There should be no instances of single classes in a day or too many of which.<br>
      2. Room non-conflict (A room cannot hold two courses at the same time
    </td>
    <td>Maximize</td>
    <td>High</td>
  </tr>
  <tr>
    <td>
      SPREAD of a TEACHER's allocation throughout all available times<br>
      <br>
      Appeases:<br>
      Teacher non-conflict (A teacher cannot hold two courses at the same time
    </td>
    <td>Maximize</td>
    <td>High</td>
  </tr>
</table>

Dimensions:
- course
- timeslot
- venue

<table>
  <tr><th>Heuristic</th><th>Depends on:</th><th>PRUNE</th></tr>
  <tr><td>$Preference_{time}$</td><td>= Weight of a timeslot</td></tr>
  <tr><td>$Quota$</td><td>= room capacity - course capacity (if positive)</td><td>(if negative)</td></tr>
  <tr><td>$Spread_{venue}$</td><td>= number of available timeslots in a day</td></tr>
  <tr><td>$Spread_{teacher}$</td><td>= number of available timeslots in a day</td></tr>
</table>

In [106]:
timeslots_dict = {
    "Sun": {
        "8-10":0,"10-12":0,"12-14":0,"14-16":0,"16-18":0,"18-20":0
    }, "Mon": {
        "8-10":3,"10-12":3,"12-14":2,"14-16":3,"16-18":2,"18-20":1
    }, "Tue": {
        "8-10":3,"10-12":3,"12-14":2,"14-16":3,"16-18":2,"18-20":1
    }, "Wed": {
        "8-10":3,"10-12":3,"12-14":2,"14-16":1,"16-18":1,"18-20":1
    }, "Thu": {
        "8-10":3,"10-12":3,"12-14":2,"14-16":3,"16-18":2,"18-20":1
    }, "Fri": {
        "8-10":3,"10-12":3,"12-14":0,"14-16":3,"16-18":2,"18-20":1
    }, "Sat": {
        "8-10":0,"10-12":0,"12-14":0,"14-16":0,"16-18":0,"18-20":0
    }
}
teachers_courses_dict = {
    "Mr M": {
        "unavailable": {},
        "courses": {
          "Mathematical Methods": {"equipment": None, "quota": 60}}},
    "Mr L": {
        "unavailable": {"Mon":["8-10"],"Tue":["8-10"]},
        "courses": {
          "Linguistics 1": {"equipment": None, "quota": 60},
          "Linguistics 2": {"equipment": None, "quota": 60}}},
    "Dr B": {
        "unavailable": {"Fri":["All"]},
        "courses": {
          "Biology 1": {"equipment": "bio", "quota": 20},
          "Biology 2": {"equipment": "bio", "quota": 20}}},
    "Dr C": {
        "unavailable": {},
        "courses": {
          "Chemistry 1": {"equipment": "chem", "quota": 30},
          "Chemistry 2": {"equipment": "chem", "quota": 30}}}
}
venues_dict = {
    "Lecture Theater 1": {"equipment": None, "quota": 100},
    "Mini Theater 1": {"equipment": None, "quota": 50},
    "Bio Lab 1": {"equipment": "bio", "quota": 30},
    "Bio Lab 2": {"equipment": "bio", "quota": 30},
    "Chem Lab 1": {"equipment": "chem", "quota": 30},
    "Chem Lab 2": {"equipment": "chem", "quota": 30},
}

s_data = timeslots_dict
c_data = teachers_courses_dict
v_data = venues_dict

class Timeslot:
  def __init__(self, id, day, timerange, weight):
    self.id = id
    self.day = day
    self.timerange = timerange
    self.weight = weight
  def __str__(self):
    return f"[{self.id}]{self.day}:{self.timerange}(W={self.weight})"

class Timeslots:
  # This class stores references to non-duplicate Timeslot objects
  def __init__(self, data_dict):
    self.data_dict = data_dict
    self.all = []
    self.available = []
    self.byDay = {}
    count = 0
    for day in data_dict:
      self.byDay[day] = []
      for slot in data_dict[day]:
        weight = data_dict[day][slot]
        obj = Timeslot(count,day,slot,weight)
        count+=1

        self.all.append(obj)
        self.byDay[day].append(obj)
        if weight>0:
          self.available.append(obj)

  def disp(self, criteria="all"):
    arr = self.get(criteria)

    for obj in arr:
      print(str(obj))

  def get(self, criteria="all", timerange="All"):
    if criteria=="all":
      return self.all
    elif criteria=="available":
      return self.available
    else:
      slots = self.byDay[criteria]
      if timerange=="All":
        return slots
      else:
        for obj in slots:
          if obj.timerange == timerange:
            return obj
        return None

timeslots = Timeslots(s_data)

class Teacher:
  def __init__(self, id, name, available=timeslots.available):
    self.available = [] + available
    self.id = id
    self.name = name

  def avail(self, newslot):
    if newslot not in self.available:
      self.available.append(newslot)

  def unavail(self, slot):
    if slot in self.available:
      self.available.remove(slot)

  def availWholeDay(self, day):
    slots = timeslots.byDay[day]
    for slot in slots:
      self.avail(slot)

  def unavailWholeDay(self, day):
    slots = timeslots.byDay[day]
    for slot in slots:
      self.unavail(slot)

  def dispAvailability(self):
    for obj in self.available:
      print(str(obj))

class Teachers:
  # This class stores references to non-duplicate Teacher objects
  def __init__(self, data_dict):
    self.data_dict = data_dict
    self.all = []
    count = 0
    for teacherName in data_dict:
      teacherObj = Teacher(count, teacherName)
      for day in data_dict[teacherName]["unavailable"]:
        for timerange in data_dict[teacherName]["unavailable"][day]:
          if timerange=="All":
            teacherObj.unavailWholeDay(day)
          else:
            slot = timeslots.get(day, timerange)
            teacherObj.unavail(slot)
      self.all.append(teacherObj)
      count+=1

  def get(self, teacherName):
    for teacher in self.all:
      if teacher.name == teacherName:
        return teacher
    return None

teachers = Teachers(c_data)

class Course:
  def __init__(self, id, name, equipment, quota, teacher):
    self.id = id
    self.name = name
    self.equipment = equipment
    self.quota = quota
    self.teacher = teacher

  def __str__(self):
    return f"[{self.id}|{self.name}|{self.teacher.name}|E={self.equipment}|Q={self.quota}]"

class Courses:
  def __init__(self, data_dict):
    self.data_dict = data_dict
    self.all = []
    self.byTeacher = {}
    count = 0
    for teacherName in data_dict:
      self.byTeacher[teacherName] = []
      teacher = teachers.get(teacherName)
      teachings = data_dict[teacherName]["courses"]
      for courseName in teachings:
        equipment = teachings[courseName]["equipment"]
        quota = teachings[courseName]["quota"]
        obj = Course(count, courseName, equipment, quota, teacher)
        self.all.append(obj)
        self.byTeacher[teacherName].append(obj)
        count+=1

  def disp(self, criteria="all"):
    arr = self.get(criteria)

    for obj in arr:
      print(str(obj))

  def get(self, criteria="all"):
    if criteria=="all":
      return self.all
    else:
      return self.byTeacher[criteria]

courses = Courses(c_data)

class Venue:
  def __init__(self, id, name, equipment, quota, available = timeslots.available):
    self.available = [] + available
    self.id = id
    self.name = name
    self.equipment = equipment
    self.quota = quota

  def avail(self, newslot):
    if newslot not in self.available:
      self.available.append(newslot)

  def unavail(self, slot):
    if slot in self.available:
      self.available.remove(slot)

  def availWholeDay(self, day):
    slots = timeslots.byDay[day]
    for slot in slots:
      self.avail(slot)

  def unavailWholeDay(self, day):
    slots = timeslots.byDay[day]
    for slot in slots:
      self.unavail(slot)

  def dispAvailability(self):
    for obj in self.available:
      print(str(obj))

  def __str__(self):
    return f"[{self.id}|{self.name}|E={self.equipment}|Q={self.quota}]"

class Venues:
  def __init__(self, data_dict):
    self.data_dict = data_dict
    self.all = []
    self.byEquipment = {}
    count = 0
    for venueName in data_dict:
      quota = data_dict[venueName]["quota"]
      equipment = data_dict[venueName]["equipment"]

      obj = Venue(count, venueName, equipment, quota)

      if equipment not in self.byEquipment:
        self.byEquipment[equipment] = []
      self.byEquipment[equipment].append(obj)
      self.all.append(obj)

      count+=1

  def disp(self, criteria="all"):
    arr = self.get(criteria)

    for obj in arr:
      print(str(obj))

  def get(self, criteria="all"):
    if criteria=="all":
      return self.all
    else:
      return self.byEquipment[criteria]

venues = Venues(v_data)

In [95]:
# timeslots, teachers, courses, venues

class Node:
  def __init__(self, course, timeslot, venue):
    self.children = []
    self.course = course
    self.timeslot = timeslot
    self.venue = venue

  def addChild(self, child):
    self.children.append(child)

  def addChildren(self, children):
    self.children += children


False

#Old solution

In [ ]:
# list of indices of variables
# timeslots s[i]
# teachers t[i]
# |-unavailabilities u[i]
# |-courses c[i]
#   |-equipment e[i]
#   |-quota q[i]
# venues v[i]
# |-equipment e[i]
# |-quota q[i]

timeslots_dict = {
    "Sun": {
        "8-10":0,"10-12":0,"12-14":0,"14-16":0,"16-18":0,"18-20":0
    }, "Mon": {
        "8-10":3,"10-12":3,"12-14":2,"14-16":3,"16-18":2,"18-20":1
    }, "Tue": {
        "8-10":3,"10-12":3,"12-14":2,"14-16":3,"16-18":2,"18-20":1
    }, "Wed": {
        "8-10":3,"10-12":3,"12-14":2,"14-16":1,"16-18":1,"18-20":1
    }, "Thu": {
        "8-10":3,"10-12":3,"12-14":2,"14-16":3,"16-18":2,"18-20":1
    }, "Fri": {
        "8-10":3,"10-12":3,"12-14":0,"14-16":3,"16-18":2,"18-20":1
    }, "Sat": {
        "8-10":0,"10-12":0,"12-14":0,"14-16":0,"16-18":0,"18-20":0
    }
}
days = list(timeslots_dict)
timeslots = list(timeslots_dict[days[0]])
teachers_courses_dict = {
    "Mr M": {
        "unavailable": {},
        "courses": {
          "Mathematical Methods": {"equipment": 0, "quota": 60}}},
    "Mr L": {
        "unavailable": {"All":["8-10"]},
        "courses": {
          "Linguistics 1": {"equipment": 0, "quota": 60},
          "Linguistics 2": {"equipment": 0, "quota": 60}}},
    "Dr B": {
        "unavailable": {"Fri":["All"]},
        "courses": {
          "Biology 1": {"equipment": "bio", "quota": 20},
          "Biology 2": {"equipment": "bio", "quota": 20}}},
    "Dr C": {
        "unavailable": {},
        "courses": {
          "Chemistry 1": {"equipment": "chem", "quota": 30},
          "Chemistry 2": {"equipment": "chem", "quota": 30}}}
}
teachers = list(teachers_courses_dict)
venues_dict = {
    "Lecture Theater 1": {"equipment": 0, "quota": 100},
    "Mini Theater 1": {"equipment": 0, "quota": 50},
    "Bio Lab 1": {"equipment": "bio", "quota": 30},
    "Bio Lab 2": {"equipment": "bio", "quota": 30},
    "Chem Lab 1": {"equipment": "chem", "quota": 30},
    "Chem Lab 2": {"equipment": "chem", "quota": 30},
}

s_data = timeslots_dict
c_data = teachers_courses_dict
v_data = venues_dict

valTimeslots = []
for day in s_data:
  for slot in s_data[day]:
    valTimeslots.append(s_data[day][slot])
numTimeslots = len(valTimeslots)
numTimeslotsPerDay = len(s_data[day])

numTeachers = len(c_data)
numCoursesTotal = 0
numCourses = []
valCQuotas = []
valCEquipments = []
for teacher in c_data:
  numCourses.append(len(c_data[teacher]["courses"]))
  numCoursesTotal += len(c_data[teacher]["courses"])
  for course in c_data[teacher]["courses"]:
    valCQuotas.append(c_data[teacher]["courses"][course]["quota"])
    valCEquipments.append(c_data[teacher]["courses"][course]["equipment"])

numVenues = len(v_data)
valVQuotas = []
valVEquipments = []
for venue in v_data:
  valVQuotas.append(v_data[venue]["quota"])
  valVEquipments.append(v_data[venue]["equipment"])

# Constraints
# Teachers cannot teach in two courses in one timeslot simultaneously
  # clash_f(t_i s_l c_j,t_i s_l c_k) {s_lcj == s_lck} < 1
# Teachers cannot teach in timeslots they are not available (for reasons other than teaching)
  # clash_f(t_i s_l c_j,t_i s_l u_k) {s_lcj == s_luk} < 1
# Practicals cannot be conducted in venues without proper equipment
  # clash_f(c_j e, v_i e) {e_cj == e_vi} > 0
# Courses cannot be allocated in venues with insufficient space for students
  # clash_f(c_j q, v_i q) {q_cj <= q_vi} > 0

#courses should be identified by ticj

# Preferences
# Timeslots with higher priorities should be filled first
# Preferences (to be added)
# Courses of same levels of requirement & major better be in different timeslots
# (This covers the problem of students cannot be in two courses in one timeslot)
# (It is not as strict of a problem as students should be responsible of planning their modules accordingly)
# (i.e., not postpone taking possibly clashing modules till the end of the programme)


#  maximize nothing!
#  subject to
#        t1c1s1 t1c1s2 t1c1s3 t1c1s4 t1c1s5 summed = 1
#        i.e., a "sum_k^K(ticjsk) = 1" constraint for each i j pair
#        t1u1s1 = 1
#        i.e., a "prod_n^N(tus_n)" for all tusses or just tus=1s without prod
#        t1c1s1 + t1u1s1 == 1
#        i.e., a "ticksj + tiulsj == 1" for each t : us
#        all binary


# Using pycsp3 (old solution)
Failed because too much processing going on

In [ ]:
%pip install pycsp3
import numpy as np
from pycsp3 import *
clear()

tcs = VarArray(size=[numTeachers, max(numCourses), numTimeslots],
             dom=lambda t, c, s: [0,1]
             if c < numCourses[t]
             else None)
#vs = VarArray(size=[numVenues, numTimeslots],dom=[0,1])

obj_expr = "0"
for s in range(numTimeslots):
  w = valTimeslots[s]
  obj_expr += f" + {w}*Sum(tcs[:][:][{s}])"
#maximize(eval(obj_expr))

# Teachers cannot be teaching two courses in one timeslot
c0_expr = ""
for t in range(numTeachers):
  for s in range(numTimeslots):
    c0_expr += f"Sum(tcs[{t},:,{s}]) <= 1, "

# Each course must be taught once a week
c1_expr = ""
for t in range(numTeachers):
  for c in range(numCourses[t]):
    c1_expr += f"Sum(tcs[{t},{c},:]) == 1, "

# Teachers cannot teach in timeslots they are not available (for reasons other than teaching)
c2_expr = ""
wholeDay = numTimeslotsPerDay
for teacher in c_data:
  u_daytimes = c_data[teacher]["unavailable"]
  t = teachers.index(teacher)
  for u_day in u_daytimes:
    if u_day=="All":
      for u_slot in u_daytimes[u_day]:
        if u_slot=="All":
          c2_expr += f"Sum(tcs[{t},:,:]) == 0, "
        else:
          #weeeek
          u = list(np.array([0,1,2,3,4,5,6])*numTimeslotsPerDay + timeslots.index(u_slot))
          for d in range(7):
            c2_expr += f"Sum(tcs[{t},:,{u[d]}]) == 0, "
    else:
      d = days.index(u_day)
      for u_slot in u_daytimes[u_day]:
        if u_slot=="All":
          c2_expr += f"Sum(tcs[{t},:,{d*wholeDay}:{(d+1)*wholeDay}]) == 0, "
        else:
          u = d*wholeDay + timeslots.index(u_slot)
          c2_expr += f"Sum(tcs[{t},:,{u}]) == 0, "


satisfy(
  eval(c0_expr),
#  eval(c1_expr),
#  eval(c2_expr)
#  tcs[:] == 1
);

if solve(sols=ALL) is SAT:
  print(values(tcs))
#solve(sols=ALL)